!pip install yfinance
!pip install bs4
!pip install nbformat
!pip install --upgrade plotly

In [326]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
from decimal import Decimal
import re
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [330]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [332]:
def make_graph(stock_data, revenue, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()
    from IPython.display import display, HTML
    fig_html = fig.to_html()
    display(HTML(fig_html))

In [371]:
tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")
tesla_data.reset_index(inplace=True)
print(tesla_data.head())


                       Date      Open      High       Low     Close  \
0 2010-06-29 00:00:00-04:00  1.266667  1.666667  1.169333  1.592667   
1 2010-06-30 00:00:00-04:00  1.719333  2.028000  1.553333  1.588667   
2 2010-07-01 00:00:00-04:00  1.666667  1.728000  1.351333  1.464000   
3 2010-07-02 00:00:00-04:00  1.533333  1.540000  1.247333  1.280000   
4 2010-07-06 00:00:00-04:00  1.333333  1.333333  1.055333  1.074000   

      Volume  Dividends  Stock Splits  
0  281494500        0.0           0.0  
1  257806500        0.0           0.0  
2  123282000        0.0           0.0  
3   77097000        0.0           0.0  
4  103003500        0.0           0.0  


In [369]:
# Get HTML
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data = requests.get(url).text

# Parse HTML
soup = BeautifulSoup(html_data, "html.parser")

# Find correct table
tables = soup.find_all("table")
for table in tables:
    if "Tesla Quarterly Revenue" in table.text:
        target_table = table
        break

# Extract values
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])
for row in target_table.tbody.find_all("tr"):
    cols = row.find_all("td")
    if len(cols) == 2:
        date = cols[0].text.strip()
        raw_revenue = cols[1].text.strip()

        # Use regex to extract numbers (keep decimals, commas)
        match = re.search(r'[\d\.,]+', raw_revenue)
        if match:
            number_str = match.group(0).replace(",", "")
            try:
                revenue = Decimal(number_str) * 1_000_000_000  # Multiply to convert billions
                tesla_revenue = pd.concat([
                    tesla_revenue,
                    pd.DataFrame({"Date": [date], "Revenue": [revenue]})
                ], ignore_index=True)
            except:
                continue

# Show result
print(tesla_revenue.tail())


          Date      Revenue
48  2010-09-30  31000000000
49  2010-06-30  28000000000
50  2010-03-31  21000000000
51  2009-09-30  46000000000
52  2009-06-30  27000000000


In [373]:
gme = yf.Ticker("GME")
gme_data = gme.history(period="max")
gme_data.reset_index(inplace=True)
print(gme_data.head())

                       Date      Open      High       Low     Close    Volume  \
0 2002-02-13 00:00:00-05:00  1.620129  1.693350  1.603296  1.691667  76216000   
1 2002-02-14 00:00:00-05:00  1.712707  1.716074  1.670626  1.683250  11021600   
2 2002-02-15 00:00:00-05:00  1.683250  1.687458  1.658002  1.674834   8389600   
3 2002-02-19 00:00:00-05:00  1.666418  1.666418  1.578047  1.607504   7410400   
4 2002-02-20 00:00:00-05:00  1.615921  1.662210  1.603296  1.662210   6892800   

   Dividends  Stock Splits  
0        0.0           0.0  
1        0.0           0.0  
2        0.0           0.0  
3        0.0           0.0  
4        0.0           0.0  


In [314]:
import warnings
from bs4 import XMLParsedAsHTMLWarning

warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)


In [375]:
# Get HTML
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/gme_revenue.htm"
html_data = requests.get(url).text

# Parse HTML
soup = BeautifulSoup(html_data, "html.parser")

# Find correct table
tables = soup.find_all("table")
for table in tables:
    if "GameStop Quarterly Revenue" in table.text:
        target_table = table
        break

# Extract values
gme_revenue = pd.DataFrame(columns=["Date", "Revenue"])
for row in target_table.tbody.find_all("tr"):
    cols = row.find_all("td")
    if len(cols) == 2:
        date = cols[0].text.strip()
        raw_revenue = cols[1].text.strip()

        # Use regex to extract numbers (keep decimals, commas)
        match = re.search(r'[\d\.,]+', raw_revenue)
        if match:
            number_str = match.group(0).replace(",", "")
            try:
                revenue = Decimal(number_str) * 1_000_000_000  # Multiply to convert billions
                gme_revenue = pd.concat([
                    gme_revenue,
                    pd.DataFrame({"Date": [date], "Revenue": [revenue]})
                ], ignore_index=True)
            except:
                continue

# Show result
print(gme_revenue.tail())


          Date      Revenue
48  2010-09-30  31000000000
49  2010-06-30  28000000000
50  2010-03-31  21000000000
51  2009-09-30  46000000000
52  2009-06-30  27000000000


In [316]:
import plotly.graph_objects as go

def make_graph(stock_data, revenue_data, stock_name):
    fig = go.Figure()

    # Stock price
    fig.add_trace(go.Scatter(
        x=stock_data['Date'],
        y=stock_data['Close'],
        name=f"{stock_name} Stock Price"
    ))

    # Revenue
    fig.add_trace(go.Scatter(
        x=revenue_data['Date'],
        y=revenue_data['Revenue'],
        name=f"{stock_name} Revenue",
        yaxis="y2"
    ))

    fig.update_layout(
        title=f"{stock_name} Stock Price and Revenue (up to June 2021)",
        xaxis=dict(title="Date", rangeslider=dict(visible=True)),
        yaxis=dict(title="Stock Price"),
        yaxis2=dict(
            title="Revenue",
            overlaying="y",
            side="right"
        )
    )

    fig.show()


In [339]:
make_graph(tesla_data, tesla_revenue, 'Tesla')


In [337]:
make_graph(gme_data, gme_revenue, 'GameStop')


In [377]:
import os
os.getcwd()


'C:\\Users\\erasc\\Learnings\\WebScraping\\StockData'